# Sample Workflow for d3rlpy Experiments

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import math
import subprocess
import os
import d3rlpy
plt.style.use('matplotlibrc')

from Python.data_sampler import *

## Building an MDPDataset

We first read in a large batch of samples from the file. As `d3rlpy` wants it in the form (observations, actions, rewards, terminal flags), we go ahead and do that. Here's a helper function to get a dataset from a list of chunks of your choosing.

In [2]:
def get_dataset(chunks : list, batch_size=30000, 
                path="collected_data/rl_det_small.txt") -> d3rlpy.dataset.MDPDataset :
    random.seed(0)
    samples = DataSampler(path_to_data=path)
    states = []
    actions = []
    rewards = []
    next_states = []
    for chunk in chunks:
        samples.use_chunk(chunk)
        samples.read_chunk()
        [statesChunk, actionsChunk, rewardsChunk, nextStatesChunk] = samples.get_batch(batch_size)
        states.append(statesChunk)
        actions.append(actionsChunk)
        rewards.append(rewardsChunk)
        next_states.append(nextStatesChunk)
    states = torch.cat(states)
    actions = torch.cat(actions)
    rewards = torch.cat(rewards)
    next_states = torch.cat(next_states)
    terminals = np.zeros(len(states))
    terminals[::100] = 1 #episode length 100, change if necessary
    print(states.shape)
    dataset = d3rlpy.dataset.MDPDataset(states.numpy(), 
                                        actions.numpy(), 
                                        rewards.numpy(), terminals)
    return dataset, states.numpy(), actions.numpy(), rewards.numpy()

We can build the dataset from there, just like this, and split into train and test sets.

In [3]:
dataset, states, actions, rewards = get_dataset([i for i in range(998)], path="collected_data/rl_deterministic.txt")

start
[ 0.00000000e+00  7.95731469e+08 -7.69891077e-02  4.00000469e-03
  5.99998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.50295370e-01 -2.41931634e-01  6.00000000e-01]
Read chunk # 1 out of 10000
start
[ 0.00000000e+00  7.95731469e+08  1.47410892e-01 -8.79999531e-03
  2.29998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  9.86362318e-02  3.75821673e-01 -6.00000000e-01]
Read chunk # 2 out of 10000
start
[ 0.00000000e+00  7.95731469e+08 -1.03891077e-02 -1.41999953e-02
 -2.10001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.78778459e-03 -1.34615461e-02  4.84073546e-02]
Read chunk # 3 out of 10000
start
[ 0.00000000e+00  7.95731469e+08 -8.17891077e-02 -1.19999531e-03
  7.39998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.09713430e-01 -2.63658359e-01  6.00000000e-01]
Read chunk # 4 out of 10000
start
[ 0.00000000e+00  7.95731469e+08 -7.24891077e-02 -1.35999953e-02
 -4.20001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.2331

[ 0.00000000e+00  7.95731469e+08 -1.24891077e-02  6.40000469e-03
  1.59998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.89056008e-02 -4.28655416e-02  1.34176685e-01]
Read chunk # 41 out of 10000
start
[ 0.00000000e+00  7.95731469e+08  1.35110892e-01  6.00004692e-04
  3.19998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  8.62448158e-02  3.37467545e-01 -6.00000000e-01]
Read chunk # 42 out of 10000
start
[ 0.00000000e+00  7.95731469e+08 -1.25889108e-01  8.60000469e-03
  4.49998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  9.04897050e-02 -3.61685289e-01  6.00000000e-01]
Read chunk # 43 out of 10000
start
[ 0.00000000e+00  7.95731469e+08  4.54108923e-02  9.20000469e-03
  7.79998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.79504108e-01  7.04170084e-02 -3.91330857e-01]
Read chunk # 44 out of 10000
start
[ 0.00000000e+00  7.95731469e+08 -1.00989108e-01 -1.45999953e-02
 -3.90001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -5.005237

[ 0.00000000e+00  7.95731469e+08 -1.41789108e-01 -1.99995308e-04
  6.79998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.89125937e-01 -4.18590168e-01  6.00000000e-01]
Read chunk # 82 out of 10000
start
[ 0.00000000e+00  7.95731469e+08  6.10108923e-02  1.32000047e-02
  7.09998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.44237123e-01  1.16216092e-01 -5.22459650e-01]
Read chunk # 83 out of 10000
start
[ 0.00000000e+00  7.95731469e+08  1.11710892e-01 -1.49999953e-02
 -7.20001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.39860767e-01  3.42562745e-01 -6.00000000e-01]
Read chunk # 84 out of 10000
start
[ 0.00000000e+00  7.95731469e+08 -1.99891077e-02 -1.91999953e-02
  3.09998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.62080602e-01 -7.23826969e-02  1.20921962e-01]
Read chunk # 85 out of 10000
start
[ 0.00000000e+00  7.95731469e+08  1.06108923e-02 -1.19999531e-03
 -9.60001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.611164

[ 0.00000000e+00  7.95731469e+08  4.60108923e-02  1.68000047e-02
  6.19998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.05018950e-01  8.23167968e-02 -3.72379504e-01]
Read chunk # 121 out of 10000
start
[ 0.00000000e+00  7.95731469e+08 -3.10891077e-02 -1.15999953e-02
  3.39998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.40217605e-01 -1.03693925e-01  2.47887223e-01]
Read chunk # 122 out of 10000
start
[ 0.00000000e+00  7.95731469e+08 -3.82891077e-02  1.34000047e-02
 -3.40001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.47363548e-01 -7.89246193e-02  3.94918266e-01]
Read chunk # 123 out of 10000
start
[ 0.00000000e+00  7.95731469e+08 -5.68891077e-02 -5.79999531e-03
 -1.00000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.53932923e-01 -8.56160479e-02  5.04761879e-01]
Read chunk # 124 out of 10000
start
[ 0.00000000e+00  7.95731469e+08 -1.37889108e-01  1.74000047e-02
 -7.70001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.82

[ 0.00000000e+00  7.95731469e+08 -1.15389108e-01  2.20000469e-03
 -3.10001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -9.45911398e-02 -2.84914456e-01  6.00000000e-01]
Read chunk # 161 out of 10000
start
[ 0.00000000e+00  7.95731469e+08 -4.69891077e-02 -1.81999953e-02
  9.99986580e-04  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  9.99492982e-02 -1.30307889e-01  3.73407266e-01]
Read chunk # 162 out of 10000
start
[ 0.00000000e+00  7.95731469e+08 -1.09891077e-02 -8.79999531e-03
  5.79998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.95571791e-01 -6.59640509e-02  7.13476936e-02]
Read chunk # 163 out of 10000
start
[ 0.00000000e+00  7.95731469e+08  5.77108923e-02 -1.45999953e-02
  6.99998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.51832656e-01  1.08126650e-01 -5.81577960e-01]
Read chunk # 164 out of 10000
start
[ 0.00000000e+00  7.95731469e+08 -1.33891077e-02  1.40000047e-02
  8.99998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.93

...

[ 0.00000000e+00  7.95731469e+08 -4.30891077e-02  4.69199998e-09
 -4.90001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.35709783e-01 -8.19612047e-02  3.96050998e-01]
Read chunk # 819 out of 10000
start
[ 0.00000000e+00  7.95731469e+08 -2.95891077e-02 -1.87999953e-02
 -6.90001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.16466125e-01 -3.33417381e-02  2.10837734e-01]
Read chunk # 820 out of 10000
start
[ 0.00000000e+00  7.95731469e+08  1.42310892e-01 -8.59999531e-03
 -1.90001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.84804326e-02  3.89392258e-01 -6.00000000e-01]
Read chunk # 821 out of 10000
start
[ 0.00000000e+00  7.95731469e+08 -6.48910771e-03  4.40000469e-03
  1.19998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.57671880e-02 -2.44077179e-02  7.23400618e-02]
Read chunk # 822 out of 10000
start
[ 0.00000000e+00  7.95731469e+08  9.58108923e-02  5.80000469e-03
  8.29998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.06

[ 0.00000000e+00  7.95731469e+08  1.09010892e-01  1.94000047e-02
  7.19998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.22392906e-01  2.42610592e-01 -6.00000000e-01]
Read chunk # 859 out of 10000
start
[ 0.00000000e+00  7.95731469e+08 -1.02489108e-01 -1.39999531e-03
  4.59998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.32958976e-01 -3.00398270e-01  6.00000000e-01]
Read chunk # 860 out of 10000
start
[ 0.00000000e+00  7.95731469e+08 -1.20489108e-01 -2.79999531e-03
  5.39998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.60673631e-01 -3.53193791e-01  6.00000000e-01]
Read chunk # 861 out of 10000
start
[ 0.00000000e+00  7.95731469e+08  1.49810892e-01  1.86000047e-02
 -5.50001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.26168719e-01  4.32443653e-01 -6.00000000e-01]
Read chunk # 862 out of 10000
start
[ 0.00000000e+00  7.95731469e+08  4.18108923e-02 -1.65999953e-02
  5.39998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.15

[ 0.00000000e+00  7.95731469e+08 -1.34589108e-01 -6.39999531e-03
  7.99986580e-04  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  4.75644967e-02 -3.60865066e-01  6.00000000e-01]
Read chunk # 897 out of 10000
start
[ 0.00000000e+00  7.95731469e+08  1.38110892e-01  2.00004692e-04
 -9.40001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.61135209e-01  4.26612914e-01 -6.00000000e-01]
Read chunk # 898 out of 10000
start
[ 0.00000000e+00  7.95731469e+08  4.87108923e-02  2.00004692e-04
  2.79998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  7.67544535e-02  1.11375317e-01 -4.50798258e-01]
Read chunk # 899 out of 10000
start
[ 0.00000000e+00  7.95731469e+08 -1.10889108e-01  8.80000469e-03
 -1.30001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -7.09403578e-02 -2.84605327e-01  6.00000000e-01]
Read chunk # 900 out of 10000
start
[ 0.00000000e+00  7.95731469e+08  5.92108923e-02 -1.17999953e-02
 -1.50001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  5.30

[ 0.00000000e+00  7.95731469e+08 -8.62891077e-02 -1.99999953e-02
  2.19998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.40330290e-01 -2.42054911e-01  6.00000000e-01]
Read chunk # 935 out of 10000
start
[ 0.00000000e+00  7.95731469e+08  9.79108923e-02  4.00004692e-04
 -8.10001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.25924567e-01  3.11839424e-01 -6.00000000e-01]
Read chunk # 936 out of 10000
start
[ 0.00000000e+00  7.95731469e+08  7.87108923e-02  1.30000047e-02
 -7.00013420e-04  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -7.09967005e-02  2.13331749e-01 -6.00000000e-01]
Read chunk # 937 out of 10000
start
[ 0.00000000e+00  7.95731469e+08 -9.46891077e-02 -1.59999531e-03
 -2.20001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -5.45789734e-02 -2.35929281e-01  6.00000000e-01]
Read chunk # 938 out of 10000
start
[ 0.00000000e+00  7.95731469e+08  6.58108923e-02 -6.59999531e-03
  8.19998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.53

[ 0.00000000e+00  7.95731469e+08 -2.28910771e-03  1.08000047e-02
  1.39998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -4.10143476e-03 -1.45807816e-02  5.41894402e-02]
Read chunk # 974 out of 10000
start
[ 0.00000000e+00  7.95731469e+08 -9.55891077e-02  7.00000469e-03
 -7.00001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.21660896e-01 -2.07375859e-01  6.00000000e-01]
Read chunk # 975 out of 10000
start
[ 0.00000000e+00  7.95731469e+08 -4.39891077e-02  1.18000047e-02
  2.79998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  3.07028205e-02 -1.33968728e-01  4.42384526e-01]
Read chunk # 976 out of 10000
start
[ 0.00000000e+00  7.95731469e+08 -1.18689108e-01  1.36000047e-02
  1.89998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.36940919e-03 -3.25872765e-01  6.00000000e-01]
Read chunk # 977 out of 10000
start
[ 0.00000000e+00  7.95731469e+08 -4.12891077e-02  1.36000047e-02
  9.39998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.06

In [4]:
print("The behavior policy value statistics are:")
dataset.compute_stats()['return']

The behavior policy value statistics are:


{'mean': -4.2979746,
 'std': 2.9650762,
 'min': -19.356209,
 'max': 0.0,
 'histogram': (array([   21,    43,    95,   379,  1195,  1979,  2612,  3300,  4137,
          5446,  7025,  8735, 11719, 18487, 25193, 33395, 40758, 54770,
         46735, 11121]),
  array([-19.356209  , -18.3884    , -17.420588  , -16.452778  ,
         -15.484967  , -14.517157  , -13.549346  , -12.581535  ,
         -11.613726  , -10.645915  ,  -9.678104  ,  -8.710294  ,
          -7.7424836 ,  -6.774673  ,  -5.806863  ,  -4.839052  ,
          -3.8712418 ,  -2.9034314 ,  -1.9356209 ,  -0.96781045,
           0.        ], dtype=float32))}

In [5]:
# plt.plot(states[:,3])

In [6]:
# plt.plot(actions)

In [7]:
# plt.plot(model.predict(np.array(states)))

In [8]:
from sklearn.model_selection import train_test_split
train_episodes, test_episodes = train_test_split(dataset, test_size=0.2)

## Setting up an Algorithm

In [9]:
from d3rlpy.algos import CQL
from d3rlpy.models.encoders import VectorEncoderFactory

from d3rlpy.preprocessing import MinMaxActionScaler
action_scaler = MinMaxActionScaler(minimum=-0.6, maximum=0.6)
#cql = CQL(action_scaler=action_scaler)

actor_encoder = VectorEncoderFactory(hidden_units=[12, 24, 36, 24, 12],
                                       activation='relu', use_batch_norm=True, dropout_rate=0.2)
critic_encoder = VectorEncoderFactory(hidden_units=[12, 24, 24, 12],
                                       activation='relu', use_batch_norm=True, dropout_rate=0.2)

model = CQL(q_func_factory='qr', #qr -> quantile regression q function, but you don't have to use this
            reward_scaler='standard',
            actor_encoder_factory = actor_encoder,
            critic_encoder_factory = critic_encoder,
            action_scaler=action_scaler,
          actor_learning_rate=1e-5, 
          critic_learning_rate=0.0003, 
            use_gpu=True) #change it to true if you have one
model.build_with_dataset(dataset)

In [10]:
from d3rlpy.metrics.scorer import td_error_scorer
from d3rlpy.metrics.scorer import average_value_estimation_scorer
from d3rlpy.metrics.scorer import initial_state_value_estimation_scorer

# calculate metrics with test dataset
ave_error_init = average_value_estimation_scorer(model, test_episodes)
print(ave_error_init)

-0.019474013344166884


In [11]:
#%load_ext tensorboard
#%tensorboard --logdir runs

In [12]:
model.fit(train_episodes,
        eval_episodes=test_episodes,
        n_epochs=10, 
        tensorboard_dir='runs',
        scorers={
            'td_error': td_error_scorer,
            'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer
        })

2022-04-15 19:48.22 [debug    ] RoundIterator is selected.
2022-04-15 19:48.22 [info     ] Directory is created at d3rlpy_logs/CQL_20220415194822
2022-04-15 19:48.22 [debug    ] Fitting action scaler...       action_scaler=min_max
2022-04-15 19:48.22 [debug    ] Fitting reward scaler...       reward_scaler=standard
2022-04-15 19:48.23 [warning  ] Skip building models since they're already built.
2022-04-15 19:48.23 [info     ] Parameters are saved to d3rlpy_logs/CQL_20220415194822/params.json params={'action_scaler': {'type': 'min_max', 'params': {'minimum': array(-0.6), 'maximum': array(0.6)}}, 'actor_encoder_factory': {'type': 'vector', 'params': {'hidden_units': [12, 24, 36, 24, 12], 'activation': 'relu', 'use_batch_norm': True, 'dropout_rate': 0.2, 'use_dense': False}}, 'actor_learning_rate': 1e-05, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'alpha_learning_rate': 0.0001, 'alpha_optim_factory': {'optim_cls

Epoch 1/10:   0%|          | 0/85741 [00:00<?, ?it/s]

2022-04-15 20:27.40 [info     ] CQL_20220415194822: epoch=1 step=85741 epoch=1 metrics={'time_sample_batch': 0.00031275933899537264, 'time_algorithm_update': 0.020723903086432908, 'temp_loss': 0.5277301647774326, 'temp': 0.1243415923763854, 'alpha_loss': -19313.581439918868, 'alpha': 1318.5249205411626, 'critic_loss': 17695.369889204558, 'actor_loss': 2.185401217090781, 'time_step': 0.02453802335468208, 'td_error': 1.3116502787876172, 'init_value': -5.9981889724731445, 'ave_value': -5.998183777200564} step=85741
2022-04-15 20:27.40 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220415194822/model_85741.pt


Epoch 2/10:   0%|          | 0/85741 [00:00<?, ?it/s]

2022-04-15 21:07.40 [info     ] CQL_20220415194822: epoch=2 step=171482 epoch=2 metrics={'time_sample_batch': 0.0003043265410616692, 'time_algorithm_update': 0.02055568481845553, 'temp_loss': 0.00015883790537005648, 'temp': 0.0008094566549346069, 'alpha_loss': -8375901.8132385975, 'alpha': 640866.8451333335, 'critic_loss': 7871029.914287469, 'actor_loss': 10.528318059189855, 'time_step': 0.025112815303733306, 'td_error': 2.360244169918384, 'init_value': -11.755678176879883, 'ave_value': -11.755678390894605} step=171482
2022-04-15 21:07.40 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220415194822/model_171482.pt


...

Epoch 9/10:   0%|          | 0/85741 [00:00<?, ?it/s]

2022-04-16 01:50.03 [info     ] CQL_20220415194822: epoch=9 step=771669 epoch=9 metrics={'time_sample_batch': 0.00030707452922892274, 'time_algorithm_update': 0.020735845929954203, 'temp_loss': -1.128560036883189e-06, 'temp': 3.86948897100545e-05, 'alpha_loss': -12239177.316546343, 'alpha': 1001080.3125, 'critic_loss': 12065161.480050385, 'actor_loss': 10.028992998721105, 'time_step': 0.025429588701093762, 'td_error': 1.7640987986467795, 'init_value': -9.013802528381348, 'ave_value': -9.013804238063573} step=771669
2022-04-16 01:50.03 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220415194822/model_771669.pt


Epoch 10/10:   0%|          | 0/85741 [00:00<?, ?it/s]

2022-04-16 02:30.34 [info     ] CQL_20220415194822: epoch=10 step=857410 epoch=10 metrics={'time_sample_batch': 0.0003076755517096982, 'time_algorithm_update': 0.020737744080086183, 'temp_loss': 4.55514840591495e-08, 'temp': 5.123224858212597e-05, 'alpha_loss': -12257118.904223183, 'alpha': 1001080.3125, 'critic_loss': 12076661.861279901, 'actor_loss': 7.639729850049536, 'time_step': 0.025443260273266214, 'td_error': 1.3771597688444286, 'init_value': -6.577911853790283, 'ave_value': -6.577912109429451} step=857410
2022-04-16 02:30.34 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220415194822/model_857410.pt


[(1,
  {'time_sample_batch': 0.00031275933899537264,
   'time_algorithm_update': 0.020723903086432908,
   'temp_loss': 0.5277301647774326,
   'temp': 0.1243415923763854,
   'alpha_loss': -19313.581439918868,
   'alpha': 1318.5249205411626,
   'critic_loss': 17695.369889204558,
   'actor_loss': 2.185401217090781,
   'time_step': 0.02453802335468208,
   'td_error': 1.3116502787876172,
   'init_value': -5.9981889724731445,
   'ave_value': -5.998183777200564}),
 (2,
  {'time_sample_batch': 0.0003043265410616692,
   'time_algorithm_update': 0.02055568481845553,
   'temp_loss': 0.00015883790537005648,
   'temp': 0.0008094566549346069,
   'alpha_loss': -8375901.8132385975,
   'alpha': 640866.8451333335,
   'critic_loss': 7871029.914287469,
   'actor_loss': 10.528318059189855,
   'time_step': 0.025112815303733306,
   'td_error': 2.360244169918384,
   'init_value': -11.755678176879883,
   'ave_value': -11.755678390894605}),
 (3,
  {'time_sample_batch': 0.00030402676114102704,
   'time_algorithm

In [15]:
model.save_policy("cqlDet998.pt") 

/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/d3rlpy/preprocessing/action_scalers.py:204: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  minimum = torch.tensor(
/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/d3rlpy/preprocessing/action_scalers.py:207: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  maximum = torch.tensor(


In [16]:
model.save_model("cqlDet998model.pt")

## Off-Policy Evaluation

We do get some metrics on a test set of initial state value and average value. However, these estimates (using the critic's Q-function) of model performance are biased. They're useful for validation during training, but not much else. Instead, we fit a Q-function to the data (or a separate dataset, as I've done here) separately and evaluate the model's performance on it.

Feel free to change the chunks and number of steps.

In [13]:
from d3rlpy.ope import FQE
# metrics to evaluate with
from d3rlpy.metrics.scorer import soft_opc_scorer


ope_dataset, states_ope, actions_ope, rewards_ope = get_dataset([2,4,6,8], path="collected_data/rl_deterministic.txt") #change if you'd prefer different chunks
ope_train_episodes, ope_test_episodes = train_test_split(ope_dataset, test_size=0.2)

fqe = FQE(algo=model, action_scaler = action_scaler, use_gpu=False) #change this if you have one!
fqe.fit(ope_train_episodes,
        eval_episodes=ope_test_episodes,
        tensorboard_dir='runs',
        n_epochs=50, n_steps_per_epoch=1000, #change if overfitting/underfitting
        scorers={
           'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer,
           'soft_opc': soft_opc_scorer(return_threshold=0)
        })

start
[ 0.00000000e+00  7.95731469e+08 -1.03891077e-02 -1.41999953e-02
 -2.10001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.78778459e-03 -1.34615461e-02  4.84073546e-02]
Read chunk # 3 out of 10000
start
[ 0.00000000e+00  7.95731469e+08 -7.24891077e-02 -1.35999953e-02
 -4.20001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.23311010e-02 -1.64283998e-01  6.00000000e-01]
Read chunk # 5 out of 10000
start
[ 0.00000000e+00  7.95731469e+08  7.01089229e-03 -4.19999531e-03
  7.39998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.21623335e-01 -2.86362315e-02 -8.00043364e-02]
Read chunk # 7 out of 10000
start
[ 0.00000000e+00  7.95731469e+08 -1.03989108e-01 -1.37999953e-02
  7.99998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.76352555e-01 -3.26280816e-01  6.00000000e-01]
Read chunk # 9 out of 10000
torch.Size([111080, 6])
2022-04-16 02:30.35 [debug    ] RoundIterator is selected.
2022-04-16 02:30.35 [info     ] Directory is created at d3rlp

Epoch 1/50:   0%|          | 0/878 [00:00<?, ?it/s]

/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


2022-04-16 02:30.38 [info     ] FQE_20220416023035: epoch=1 step=878 epoch=1 metrics={'time_sample_batch': 0.00011914167425898593, 'time_algorithm_update': 0.0029855746614634313, 'loss': 0.0002461599849627247, 'time_step': 0.003155057142427136, 'init_value': -0.22843682765960693, 'ave_value': -0.22831709708927517, 'soft_opc': nan} step=878


2022-04-16 02:30.38 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220416023035/model_878.pt


...

2022-04-16 02:34.22 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220416023035/model_43022.pt


Epoch 50/50:   0%|          | 0/878 [00:00<?, ?it/s]

2022-04-16 02:34.28 [info     ] FQE_20220416023035: epoch=50 step=43900 epoch=50 metrics={'time_sample_batch': 0.0001511989258959516, 'time_algorithm_update': 0.00497674344612417, 'loss': 0.029767772812324636, 'time_step': 0.005192360193691384, 'init_value': -1.7842823266983032, 'ave_value': -1.7847246024878274, 'soft_opc': nan} step=43900


2022-04-16 02:34.28 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220416023035/model_43900.pt


[(1,
  {'time_sample_batch': 0.00011914167425898593,
   'time_algorithm_update': 0.0029855746614634313,
   'loss': 0.0002461599849627247,
   'time_step': 0.003155057142427136,
   'init_value': -0.22843682765960693,
   'ave_value': -0.22831709708927517,
   'soft_opc': nan}),
 (2,
  {'time_sample_batch': 0.0001231980486719918,
   'time_algorithm_update': 0.0030236969233102297,
   'loss': 0.0014282974988773878,
   'time_step': 0.0031988797263838437,
   'init_value': -0.5504694581031799,
   'ave_value': -0.5503293672049991,
   'soft_opc': nan}),
 (3,
  {'time_sample_batch': 0.000123508698847951,
   'time_algorithm_update': 0.0031934004438222132,
   'loss': 0.004029400426500787,
   'time_step': 0.003368657922418894,
   'init_value': -0.787994921207428,
   'ave_value': -0.7878976750239928,
   'soft_opc': nan}),
 (4,
  {'time_sample_batch': 0.000124397473346128,
   'time_algorithm_update': 0.0031993448870057126,
   'loss': 0.007525220854168146,
   'time_step': 0.003378618822553978,
   'init_v

In [14]:
from d3rlpy.ope import FQE
# metrics to evaluate with
from d3rlpy.metrics.scorer import soft_opc_scorer


# ope_dataset, states_ope, actions_ope, rewards_ope = get_dataset([2,4,6,8], path="collected_data/rl_stoch_small.txt") #change if you'd prefer different chunks
# ope_train_episodes, ope_test_episodes = train_test_split(ope_dataset, test_size=0.2)

# fqe = FQE(algo=model, action_scaler = action_scaler, use_gpu=False) #change this if you have one!
# fqe.fit(ope_train_episodes, eval_episodes=ope_test_episodes,
#         tensorboard_dir='runs',
#         n_epochs=50, n_steps_per_epoch=1000, #change if overfitting/underfitting
#         scorers={
#            'init_value': initial_state_value_estimation_scorer,
#             'ave_value': average_value_estimation_scorer,
#            'soft_opc': soft_opc_scorer(return_threshold=0)
#         })